# **Github project download**

In [1]:
## 사본만들어야 수정 가능
## 전처리는 되어있으나 형태소 분석 x o
## 어제 배운거로 형태소 분석 해보면 됨
import os.path

if os.path.isdir('./korean_word_embedding_tutorial') == 'True':  
  !rm -r ./korean_word_embedding_tutorial
  !git clone https://github.com/sseol11/korean_word_embedding_tutorial
else: 
  !git clone https://github.com/sseol11/korean_word_embedding_tutorial

Cloning into 'korean_word_embedding_tutorial'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 41 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (41/41), done.


In [0]:
!mkdir korean_word_embedding_tutorial/training_data

In [0]:
!rm korean_word_embedding_tutorial/training_data

# **Training data download**

In [12]:
#!wget --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=1SaIMXjnsCHakT7sDo2ATaUUZb1k9x3No' -O pos_remove_wiki_ko.txt
## 구글드라이브는 위에 html코드가 붙어서 전처리 작업이 필요하다
!wget "https://www.dropbox.com/s/ozcb1b0mwlp4g2h/pos_remove_wiki_ko_1000k.txt?dl=0" -O pos_remove_wiki_ko.txt
!mv pos_remove_wiki_ko.txt ./korean_word_embedding_tutorial/training_data

## 형태소 분석까지 다 된거 받음

--2019-07-30 04:30:10--  https://www.dropbox.com/s/ozcb1b0mwlp4g2h/pos_remove_wiki_ko_1000k.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ozcb1b0mwlp4g2h/pos_remove_wiki_ko_1000k.txt [following]
--2019-07-30 04:30:10--  https://www.dropbox.com/s/raw/ozcb1b0mwlp4g2h/pos_remove_wiki_ko_1000k.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca2400c3f55e1d3308800110e18.dl.dropboxusercontent.com/cd/0/inline/AlpLOFPNmGMEw1YZfMCBf_Zc0yguQoaf9iQ6fwBmw44IiZVJOIeUNZPvJw1_pJmfsZZTNFiKbpzWFe5aLpHKwPQi_-TLcL1ShxPcH0qZyJvRVQ/file# [following]
--2019-07-30 04:30:10--  https://uca2400c3f55e1d3308800110e18.dl.dropboxusercontent.com/cd/0/inline/AlpLOFPNmGMEw1YZfMCBf_Zc0yguQoaf9iQ6fwBmw44IiZVJOIeUNZPvJw1_pJmfsZZTNFiKbpzWFe5aLp

In [13]:
!wc -l ./korean_word_embedding_tutorial/training_data/pos_remove_wiki_ko.txt

## 라인이 100만줄

1000000 ./korean_word_embedding_tutorial/training_data/pos_remove_wiki_ko.txt


# Environment check

In [15]:
#no.of cores each processor is having 
!lscpu | grep 'Core(s) per socket:'

Core(s) per socket:  1


In [16]:
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'

Thread(s) per core:  2


# **1. Training**

In [0]:
from gensim.models import word2vec
import logging
## 진심 cpu

In [0]:
def main():
    # import modules & set up logging
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    # read sentences, Iterate over a file that contains sentences # 파일을 병렬적으로 읽어온다.
    sentences = word2vec.LineSentence('./korean_word_embedding_tutorial/training_data/pos_remove_wiki_ko.txt')
    
    # train word2vec, sg=1 : skip-gram // workers : parallelization  #학습 , 선험적 지식으로 파라메터 설정
    # apple 0.1 -0.5 , 0.6, ..... 이런거 단위를 300정도로
    model = word2vec.Word2Vec(sentences, window=5, sg=1, size=300, workers=2)
    
    # model save
    # cpkt 모델
    model.save("./korean_word_embedding_tutorial/wiki_ko_model.cpkt") 
    




In [0]:
def model_example():
    try:
      model = word2vec.Word2Vec.load("./korean_word_embedding_tutorial/wiki_ko_model")
      vector = model.wv['서울'] ## 서울이라는 단어에 대한 워드 벡터출력 -> 위치 잘몰라서 비주얼라이제이션
      print(vector)
    except KeyError:
      print("This word not in vocabulary")
    
    try:
      target_word = ['서울']
      print(model.wv.most_similar(positive=target_word, topn=6)) ## 서울에 가장 근접한거 6개
    
    except KeyError:
      print("This word not in vocabulary")

In [0]:
if __name__ == "__main__":
 #   main()
     model_example()

In [0]:
#  ('부산', 0.6782873272895813)  코사인 시밀러리티 부산이 값이 얼마나 유사하다 이ㅓㄴ거

# 1.1 Pretrained model (부가사항)


*  **2430k lines wikipedia korea data pretrained model**
*  **본 실습모델보다 학습량이 많은 모델**




In [0]:
!mkdir ./korean_word_embedding_tutorial/pre_trained_model
!wget "https://www.dropbox.com/sh/bvvj3c650zshaju/AADfgmvxx_A6PvaxuaE5GNQka?dl=0" -O pre_trained_model.zip
!unzip pre_trained_model.zip -d ./korean_word_embedding_tutorial/pre_trained_model/
!rm pre_trained_model.zip

## 2천4백만 이게 존나 많음
# 지금 우리가한건 100백만개 이정도

mkdir: cannot create directory ‘./korean_word_embedding_tutorial/pre_trained_model’: File exists
--2019-07-23 14:09:22--  https://www.dropbox.com/sh/bvvj3c650zshaju/AADfgmvxx_A6PvaxuaE5GNQka?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/bvvj3c650zshaju/AADfgmvxx_A6PvaxuaE5GNQka [following]
--2019-07-23 14:09:22--  https://www.dropbox.com/sh/raw/bvvj3c650zshaju/AADfgmvxx_A6PvaxuaE5GNQka
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7b17222e470c13808c323dcc3e.dl.dropboxusercontent.com/zip_by_token_key?key=AlO15CZZWtv5Pqka-YruTsrffStxz8KRrs6PI-3jEf0J3y-kwzne_AdrpjgApNNA2mifhGS-lP2fGvubN0X3V4I1ZgwPwXsUtRBL--h7P2TvteyerPIuL97F0RbdmB_NGl86eIrlivHoGQC8mzUUdys3r0Ram6esB0h42Kbzw9AeajHXgfvj7B9s6FyWtY7U9UhM73r1qQVkNYSS4O

In [0]:
# _129512_300d_tensors.bytes  
# 129512_300d_labels.tsv   비주얼라이제이션 할때 쓰는거
## https://github.com/sseol11/korean_word_embedding_tutorial 여기서 배포한거 한국어처리 이거 쓰면 된다.
##

# 2. Visualization

In [0]:
!mkdir ./korean_word_embedding_tutorial/visualization

In [0]:
from collections import Counter
from gensim.models import Word2Vec
import re

In [0]:
root_path = './korean_word_embedding_tutorial/'

def main():
    model = Word2Vec.load(root_path + 'wiki_ko_model')
    
    try:
      num_rows = len(model.vocab)
    except:
      model.vocab = model.wv.vocab
      num_rows = len(model.vocab)
      
    dim = model.vector_size
    
    global tensor_out_fn
    global labels_out_fn
    
    tensor_out_fn = root_path + 'visualization/wiki_ko_model_%d_%dd_tensors.tsv' % (num_rows, dim)
    labels_out_fn = root_path + 'visualization/wiki_ko_model_%d_%dd_labels.tsv' % (num_rows, dim)
    
    
    try:
      labels_out = open(labels_out_fn, 'w', encoding='utf-8')
    except:
      labels_out = open(labels_out_fn, 'w')
      
    labels_out.write("word\tlanguage\tcount\n") ## 바이링규얼은 언어끼리 구분해야 좋다.
    wv_list=[]
    
    counter = {}
    for wd in model.vocab:
      counter[wd] = model.vocab[wd].count
    counter = Counter(counter)
    common = counter.most_common(5000) #cpu써서 많이하면 뒤진다. 상위 5천건
    words, _ = zip(*common)
    
    for wd in words:
      ww = model[wd].tolist()
      assert dim == len(ww)
      assert '\t' not in wd
      wv_list.append(ww)
      
      try:
        labels_out.write('%s\t%s\t%s\n' % (wd, 'ko', model.vocab[wd].count))
      except:
        labels_out.write('%s\t%s\t%s\n' % (wd, 'ko', model.vocab[wd].count)).encode('utf-8')
        
    with open(tensor_out_fn, 'w') as fw:
      for i in wv_list:
        fw.write("%s\n" % (str(i).replace(', ', '\t').replace('[', '').replace(']', '')))
    
    labels_out.close()

In [28]:
if __name__ =="__main__":
    main()
    
    

2019-07-30 05:26:37,003 : INFO : loading Word2Vec object from ./korean_word_embedding_tutorial/wiki_ko_model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-07-30 05:26:37,333 : INFO : loading wv recursively from ./korean_word_embedding_tutorial/wiki_ko_model.wv.* with mmap=None
2019-07-30 05:26:37,334 : INFO : loading vectors from ./korean_word_embedding_tutorial/wiki_ko_model.wv.vectors.npy with mmap=None
2019-07-30 05:26:37,375 : INFO : setting ignored attribute vectors_norm to None
2019-07-30 05:26:37,376 : INFO : loading vocabulary recursively from ./korean_word_embedding_tutorial/wiki_ko_model.vocabulary.* with mmap=None
2019-07-30 05:26:37,378 : INFO : loading trainables re

In [0]:
## 1-코사인 시뮬러리티 = 코사인 디스턴스 작을수로 가깝다

# [Embedding projector](http://projector.tensorflow.org/)


---


![대체 텍스트](https://miro.medium.com/max/1200/1*Fat62b1ZITOFMPXTcHNkLw.jpeg)